In [39]:
import requests
from tqdm import tqdm

COIN_FLIP_NUMBER = 30
MAX_ITERATION = 30
coin_flip_draws = {}
#intial assumptions
theta_A = 0.6
theta_B = 0.5
for j in tqdm(range(MAX_ITERATION), desc = 'Calculating...'):
    for i in range(COIN_FLIP_NUMBER):
        #------------------E STEP----------------------------------------------------------
        api_url = "https://24zl01u3ff.execute-api.us-west-1.amazonaws.com/beta"
        response = requests.get(api_url)
        heads_count=0
        tails_count=0
        L_A, L_B, P_A, P_B, LH_A, LT_A, LH_B, LT_B = 0,0,0,0,0,0,0,0
        for toss in response.json()['body']:
            if toss == '0':
                tails_count +=1
            elif toss == '1':
                heads_count +=1
        L_A = (theta_A ** heads_count) * ((1-theta_A) ** (20 - heads_count)) 
        L_B = (theta_B ** heads_count) * ((1-theta_B) ** (20 - heads_count))
        P_A = L_A / (L_A + L_B)
        P_B = L_B / (L_A + L_B)

        LH_A = P_A * heads_count
        LT_A = P_A * tails_count

        LH_B = P_B * heads_count
        LT_B = P_B * tails_count
        coin_flip_draws[i] = [heads_count, tails_count, P_A, P_B, LH_A, LT_A, LH_B, LT_B]        
    sum_LH_A, sum_LT_A, sum_LH_B, sum_LT_B = 0,0,0,0
    for i in range(COIN_FLIP_NUMBER):
        sum_LH_A = sum_LH_A + coin_flip_draws[i][4]
        sum_LT_A = sum_LT_A + coin_flip_draws[i][5]
        sum_LH_B = sum_LH_B + coin_flip_draws[i][6]
        sum_LT_B = sum_LT_B + coin_flip_draws[i][7]
    #------------------M STEP-------------------------------------------------------------    
    theta_A = sum_LH_A/(sum_LH_A + sum_LT_A)
    theta_B = sum_LH_B/(sum_LH_B + sum_LT_B)
    theta_A = round(theta_A, 2)
    theta_B = round(theta_B, 2)
print(theta_A, theta_B)   

Calculating...: 100%|██████████████████████████████████████████████████████████████████| 30/30 [02:19<00:00,  4.63s/it]

0.73 0.34
